# Expand the Heat Wave and Cold Snap Libraries to Include Peak Loads and Regions Impacted


In [1]:
# Start by importing the packages we need:
import os
import warnings

import pandas as pd
import numpy as np

from datetime import timedelta


## Suppress Future Warnings


In [2]:
# Suppress future warnings:
warnings.simplefilter(action='ignore', category=FutureWarning)


## Set the Directory Structure

In [3]:
# Identify the top-level directory and the subdirectory where the data will be stored:
hw_cs_data_dir =  '/Users/burl878/Documents/Code/code_repos/gdo_climate_toolsuite_visualizations/data/'
data_output_dir = '/Users/burl878/Documents/Code/code_repos/gdo_climate_toolsuite_visualizations/data/'


## Process the Heat Wave and Cold Snap Library


In [4]:
# Define a function to add new information to the existing HW and CS libraries:
def expand_hw_cs_libraries(hw_cs: str, hw_cs_data_dir: str, data_output_dir: str):

    # Read in NERC region name file and rename some columns:
    nerc = pd.read_csv((hw_cs_data_dir + 'nerc_region_names.csv'))
    nerc.rename(columns={'number': 'NERC', 'short_name': 'Region'}, inplace=True)
    nerc['NERC'] = nerc['NERC'].apply(pd.to_numeric)
    
    if hw_cs == 'HW':
       # Read in the raw data from the heat wave library:
       temp_df = pd.read_csv((hw_cs_data_dir + 'hw_library.csv'))
    elif hw_cs == 'CS':
       # Read in the raw data from the cold snap library:
       temp_df = pd.read_csv((hw_cs_data_dir + 'cs_library.csv'))
    
    # Rename the NERC region variable and strip the "NERC" string:
    temp_df.rename(columns={'NERC_ID': 'NERC'}, inplace=True)
    temp_df['NERC'] = temp_df['NERC'].str.replace('NERC', '', regex=True)
    temp_df['NERC'] = temp_df['NERC'].apply(pd.to_numeric)
    temp_df['definition'] = temp_df['definition'].str.replace('def', '', regex=True)

    # Merge in the NERC region name:
    temp_df = temp_df.merge(nerc, on='NERC', how='left')

    # Rename the temperature variable to something more generic:
    if hw_cs == 'HW':
       temp_df.rename(columns={'highest_temperature': 'T_Max_Min'}, inplace=True)
    elif hw_cs == 'CS':
       temp_df.rename(columns={'lowest_temperature': 'T_Max_Min'}, inplace=True) 
    
    # Only keep the columns we need:
    temp_df = temp_df[['NERC','Region','definition','start_date','end_date','centroid_date','T_Max_Min','duration','spatial_coverage']].copy()
      
    # Rename the columns for consistency:
    temp_df.rename(columns={'definition': 'Def', 'start_date': 'Start', 'end_date': 'End', 'centroid_date': 'Center',
                            'duration': 'Duration', 'spatial_coverage': 'Coverage'}, inplace=True)
    
    # Set all dates to datetimes variable and extract the day of year:
    temp_df['Start'] = pd.to_datetime(temp_df['Start'])
    temp_df['End'] = pd.to_datetime(temp_df['End'])
    temp_df['Center'] = pd.to_datetime(temp_df['Center'])
    temp_df['Center_DOY'] = temp_df['Center'].dt.day_of_year

    # Add a new column with a unique row number and an empty column to the dataframe to store the unique ID:
    temp_df['Row_Number'] = np.arange(len(temp_df))
    temp_df['UID_Temp'] = np.nan

    # Make a list of all of the unique NERC regions and definitions in "temp_df":
    nerc_regions = temp_df['NERC'].unique()
    definitions = temp_df['Def'].unique()

    # Loop over the unique NERC region and definition combinations:
    for i in range(len(nerc_regions)):
        for j in range(len(definitions)):
        
            # Subset to just the data for that unique combination:
            subset_df = temp_df[(temp_df['NERC'].isin([nerc_regions[i]]) & temp_df['Def'].isin([definitions[j]]))].copy()

            # If the subset isn't empty then assign a unique ID for each event with that combination:
            if subset_df.empty == False:
               # Add a new column with a unique row number:
               subset_df['Event_Number'] = np.arange(len(subset_df)) + 1

               # Add a new column with a unique identifier:
               subset_df['UID'] = ('NERC' + subset_df['NERC'].astype(str) + '_Def' + subset_df['Def'].astype(str) + '_Event' + subset_df['Event_Number'].astype(str))

               # Loop over the subset and assign the UID back into the main dataframe:
               for row in range(len(subset_df)):
                   temp_df.loc[temp_df['Row_Number'] == subset_df['Row_Number'].iloc[row], 'UID_Temp'] = subset_df['UID'].iloc[row]

            # Clean up and move to the next combination:
            del subset_df, row

    # Rename the UID variable:
    temp_df.rename(columns={'UID_Temp': 'UID'}, inplace=True)
    
    # Make a copy of the necessary variables to output:
    output_df = temp_df[['UID', 'NERC', 'Region', 'Def', 'Start', 'End', 'Center', 'Center_DOY', 'T_Max_Min', 'Duration', 'Coverage']].copy(deep=False)
    
    # Write out the dataframe to a .csv file:
    if hw_cs == 'HW':
       output_df.to_csv((os.path.join(data_output_dir, 'hw_library_expanded.csv')), sep=',', index=False)
    if hw_cs == 'CS':
       output_df.to_csv((os.path.join(data_output_dir, 'cs_library_expanded.csv')), sep=',', index=False)
        
    return output_df


In [6]:
# Test the function:
output_df = expand_hw_cs_libraries(hw_cs = 'HW',
                                   hw_cs_data_dir = hw_cs_data_dir, 
                                   data_output_dir = data_output_dir)

output_df


,UID,NERC,Region,Def,Start,End,Center,Center_DOY,T_Max_Min,Duration,Coverage
0,NERC1_Def1_Event1,1,WECC: AZ-NM-SNV,1,1995-07-24,1995-08-11,1995-07-28,209,109.9,19,50.0
1,NERC1_Def1_Event2,1,WECC: AZ-NM-SNV,1,2017-06-16,2017-06-28,2017-06-20,171,109.0,13,45.5
2,NERC1_Def1_Event3,1,WECC: AZ-NM-SNV,1,1994-06-23,1994-07-04,1994-06-28,179,108.9,12,65.9
3,NERC1_Def1_Event4,1,WECC: AZ-NM-SNV,1,1990-06-20,1990-07-02,1990-06-26,177,108.6,13,43.2
4,NERC1_Def1_Event5,1,WECC: AZ-NM-SNV,1,2021-06-13,2021-06-22,2021-06-15,166,108.2,10,43.2
...,...,...,...,...,...,...,...,...,...,...,...
28854,NERC9_Def12_Event186,9,SERC: Delta,12,2007-01-12,2007-01-14,2007-01-13,13,69.3,3,31.5
28855,NERC9_Def12_Event187,9,SERC: Delta,12,2017-01-01,2017-01-03,2017-01-03,3,68.9,3,32.1
28856,NERC9_Def12_Event188,9,SERC: Delta,12,1995-12-16,1995-12-18,1995-12-16,350,67.5,3,43.8
28857,NERC9_Def12_Event189,9,SERC: Delta,12,1991-02-04,1991-02-06,1991-02-04,35,66.9,3,42.6
